In [2]:
import csv
import os

# ===================== Segment Tree (merge-sort tree) =====================
class SegmentTree:
    def __init__(self, prices):
        """
        prices : list of (price:int, index:int)
        The root is stored at idx = 0; children: 2*idx+1, 2*idx+2
        Each node stores a sorted list of (price, index) for that segment.
        """
        self.n = len(prices)
        if self.n == 0:
            self.tree = []
        else:
            self.tree = [[] for _ in range(4 * self.n)]
            self.build(prices, 0, self.n - 1, 0)

    def merge_sorted_lists(self, left, right):
        """Merge two sorted lists of (price, index) into one sorted list."""
        res = []
        i = 0
        j = 0
        while i < len(left) and j < len(right):
            if left[i][0] <= right[j][0]:
                res.append(left[i])
                i += 1
            else:
                res.append(right[j])
                j += 1
        # append tails
        while i < len(left):
            res.append(left[i])
            i += 1
        while j < len(right):
            res.append(right[j])
            j += 1
        return res

    def build(self, prices, l, r, idx):
        """Build node idx for segment [l, r] from prices list."""
        if l == r:
            # leaf: single element list
            self.tree[idx] = [(prices[l][0], prices[l][1])]
            return
        mid = (l + r) // 2
        self.build(prices, l, mid, 2 * idx + 1)
        self.build(prices, mid + 1, r, 2 * idx + 2)
        left_list = self.tree[2 * idx + 1]
        right_list = self.tree[2 * idx + 2]
        self.tree[idx] = self.merge_sorted_lists(left_list, right_list)

    def update(self, pos, old_price, new_price, l, r, idx):
        """
        Update the element at array index `pos`: old_price -> new_price.
        We update the current node's list (remove old pair, insert new pair in sorted order),
        then recurse to the child containing pos.
        """
        # remove any pair that exactly matches (old_price, pos)
        cur = []
        for (p, i) in self.tree[idx]:
            if not (p == old_price and i == pos):
                cur.append((p, i))

        # insert (new_price, pos) in sorted order without using sorted()
        inserted = False
        newlist = []
        for (p, i) in cur:
            if not inserted and new_price <= p:
                newlist.append((new_price, pos))
                inserted = True
            newlist.append((p, i))
        if not inserted:
            newlist.append((new_price, pos))
        self.tree[idx] = newlist

        # if leaf, stop
        if l == r:
            return

        mid = (l + r) // 2
        if pos <= mid:
            self.update(pos, old_price, new_price, l, mid, 2 * idx + 1)
        else:
            self.update(pos, old_price, new_price, mid + 1, r, 2 * idx + 2)

    def range_query(self, ql, qr, low, high, l, r, idx):
        """
        Return list of (price, index) that are within index-range [ql, qr]
        and have price in [low, high].
        """
        if ql > r or qr < l:
            return []
        if ql <= l and r <= qr:
            # node fully inside index-range -> filter by price
            out = []
            for (p, i) in self.tree[idx]:
                if low <= p <= high:
                    out.append((p, i))
            return out
        mid = (l + r) // 2
        left_res = self.range_query(ql, qr, low, high, l, mid, 2 * idx + 1)
        right_res = self.range_query(ql, qr, low, high, mid + 1, r, 2 * idx + 2)
        return left_res + right_res


# ===================== Product System =====================
class ProductSystem:
    def __init__(self, filename):
        self.filename = filename
        self.products = []   # list of dicts: {"Product_ID", "Price", "Category", "Date"}
        self.seg_tree = None
        # ensure file exists with header
        if not os.path.exists(self.filename):
            with open(self.filename, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["Product_ID", "Price", "Category", "Date"])
        # load and build
        self.load_file()
        self.build_tree()

    # ---------------- File Handling ---------------- #
    def load_file(self):
        self.products = []
        try:
            with open(self.filename, "r", newline="") as f:
                reader = csv.DictReader(f)
                # manual loop, no enumerate
                for row in reader:
                    # skip empty rows
                    if not row:
                        continue
                    pid = row.get("Product_ID", "").strip()
                    price_str = (row.get("Price", "") or "").strip()
                    category = row.get("Category", "").strip()
                    date = row.get("Date", "").strip()

                    # safe price parse: empty or non-digit -> 0
                    if price_str == "":
                        price = 0
                    else:
                        # handle possible spaces or non-digit characters gracefully
                        try:
                            price = int(price_str)
                        except ValueError:
                            # fallback: extract digits if possible, else 0
                            digits = "".join(ch for ch in price_str if ch.isdigit())
                            price = int(digits) if digits else 0

                    # only store if Product_ID present (skip malformed rows)
                    if pid != "":
                        self.products.append({
                            "Product_ID": pid,
                            "Price": price,
                            "Category": category,
                            "Date": date
                        })
        except FileNotFoundError:
            self.products = []

    def save_file(self):
        """Write back to the same CSV safely (write temp then replace)."""
        temp_file = self.filename + ".tmp"
        try:
            with open(temp_file, "w", newline="") as f:
                fieldnames = ["Product_ID", "Price", "Category", "Date"]
                writer = csv.DictWriter(f, fieldnames)
                writer.writeheader()
                # manual loop for writerows to ensure consistent ordering
                for p in self.products:
                    writer.writerow({
                        "Product_ID": p["Product_ID"],
                        "Price": p["Price"],
                        "Category": p["Category"],
                        "Date": p["Date"]
                    })
            # atomic replace
            os.replace(temp_file, self.filename)
        except PermissionError as e:
            # if permission error (file open elsewhere), inform user and remove temp file if exists
            if os.path.exists(temp_file):
                try:
                    os.remove(temp_file)
                except Exception:
                    pass
            raise PermissionError(f"Cannot write to {self.filename}. Close any programs (Excel) using it.") from e

    # ---------------- Tree Handling ---------------- #
    def build_tree(self):
        """Create the prices list (price, index) without using enumerate."""
        n = 0
        prices = []
        # manual index counter
        idx = 0
        while idx < len(self.products):
            p = self.products[idx]
            prices.append((p["Price"], idx))
            idx += 1
            n += 1
        if n == 0:
            self.seg_tree = None
        else:
            self.seg_tree = SegmentTree(prices)

    # ---------------- Product Operations ---------------- #
    def insert(self, product_id, price, category, date):
        """Insert product; if product_id exists, update existing product instead."""
        # check for existing Product_ID (manual loop)
        i = 0
        while i < len(self.products):
            if self.products[i]["Product_ID"] == product_id:
                # exists -> update
                self.products[i]["Price"] = int(price)
                self.products[i]["Category"] = category
                self.products[i]["Date"] = date
                self.save_file()
                self.build_tree()
                return
            i += 1

        # not found -> append
        new_product = {
            "Product_ID": product_id,
            "Price": int(price),
            "Category": category,
            "Date": date
        }
        self.products.append(new_product)
        self.save_file()
        self.build_tree()
        print(f"Inserted product {product_id} successfully.")

    def update_price(self, product_id, new_price):
        """Update price by Product_ID and update segment tree."""
        pos = None
        i = 0
        while i < len(self.products):
            if self.products[i]["Product_ID"] == product_id:
                pos = i
                break
            i += 1

        if pos is None:
            print(f"Product {product_id} not found.")
            return

        old_price = self.products[pos]["Price"]
        new_price_int = int(new_price)
        self.products[pos]["Price"] = new_price_int

        if self.seg_tree:
            # update merge-sort tree in-place
            self.seg_tree.update(pos, old_price, new_price_int,
                                 0, len(self.products) - 1, 0)
        else:
            # no tree -> rebuild
            self.build_tree()

        self.save_file()
        print(f"Updated price of {product_id} from {old_price} to {new_price_int}.")

    def filter_by_price(self, low, high):
        """Return list of product dicts whose Price in [low, high]."""
        if not self.seg_tree or len(self.products) == 0:
            return []
        n = len(self.products)
        res_pairs = self.seg_tree.range_query(0, n - 1, low, high, 0, n - 1, 0)
        # Map indices back to product dicts without enumerate
        out = []
        j = 0
        while j < len(res_pairs):
            (_, idx) = res_pairs[j]
            out.append(self.products[idx])
            j += 1
        return out


# ===================== Example usage =====================

ps = ProductSystem("DATA.csv")
ps100 = ProductSystem("products_100.csv")
ps500 = ProductSystem("products_500.csv")
ps1000 = ProductSystem("products_1000.csv")
ps300 = ProductSystem("products_300.csv")
ps800 = ProductSystem("products_800.csv")

print("\n🔹 Filter products between 500 and 6000:")
for prod in ps.filter_by_price(500, 6000):
    print(prod)

ps.update_price("P102", 1999)

ps.insert("P107", 3000, "Sports", "2025-07-12")

print("\n🔹 Filter products between 2000 and 4000:")
for prod in ps100.filter_by_price(2000, 4000):
    print(prod)
print("\n🔹 Filter products between 500 and 1500:")
for prod in ps500.filter_by_price(500, 1500):
    print(prod)
print("\n🔹 Filter products between 500 and 2500:")
for prod in ps1000.filter_by_price(500, 2500):
    print(prod)
print("\n🔹 Filter products between 999 and 1999:")    
for prod in ps300.filter_by_price(999, 1999):
    print(prod)
print("\n🔹 Filter products between 899 and 3999:")
for prod in ps800.filter_by_price(899, 3999):
    print(prod)






🔹 Filter products between 500 and 6000:
{'Product_ID': 'P102', 'Price': 1999, 'Category': 'Accessories', 'Date': '02-07-2025'}
{'Product_ID': 'P106', 'Price': 2500, 'Category': 'Sports', 'Date': '06-07-2025'}
{'Product_ID': 'P107', 'Price': 3000, 'Category': 'Sports', 'Date': '2025-07-12'}
{'Product_ID': 'P103', 'Price': 4999, 'Category': 'Home Kitchen', 'Date': '03-07-2025'}
Updated price of P102 from 1999 to 1999.

🔹 Filter products between 2000 and 4000:
{'Product_ID': 'P0046', 'Price': 2414, 'Category': 'Beauty', 'Date': '04-07-2025'}
{'Product_ID': 'P0042', 'Price': 2421, 'Category': 'Toys', 'Date': '31-07-2025'}
{'Product_ID': 'P0040', 'Price': 2578, 'Category': 'Stationery', 'Date': '24-07-2025'}
{'Product_ID': 'P0037', 'Price': 2720, 'Category': 'Fashion', 'Date': '11-07-2025'}
{'Product_ID': 'P0079', 'Price': 3034, 'Category': 'Books', 'Date': '28-07-2025'}
{'Product_ID': 'P0020', 'Price': 3243, 'Category': 'Sports', 'Date': '12-07-2025'}
{'Product_ID': 'P0039', 'Price': 3525